> <b>参考自：[all-agentic-architectures](https://github.com/FareedKhan-dev/all-agentic-architectures/tree/main)</b>

# Reflection

## 1. 定义
**反思架构**是指 Agent 在向用户返回最终答案之前，先对其自身的输出进行批评（Critique）和修订（Revise）的一种工作模式。

- **核心机制**：它摒弃了单次生成的模式，转而采用“生成 -> 评估 -> 改进”的多步内部独白机制。
- **类比**：这模仿了人类在处理复杂任务时的“起草、审查、编辑”过程，旨在捕捉错误并提高最终输出的质量。

---

## 2. 宏观工作流
该过程通常分为三个连续的阶段：

1. **生成 (Generate)**
   Agent 根据用户的 Prompt 产出初稿或初步解决方案。
2. **批评 (Critique)**
   Agent 切换角色成为“批评者”或“审查员”。它会进行自我反问，例如：
   - “这个答案有什么问题？”
   - “遗漏了什么关键信息？”
   - “目前的方案是最优解吗？”
   - “是否存在逻辑漏洞或代码 Bug？”
3. **优化 (Refine)**
   利用在批评阶段获得的见解，Agent 对初稿进行修改，生成最终的改进版本。

---

## 3. 应用场景
- **代码生成 (Code Generation)**
   初生成的代码可能包含 Bug、效率低下或缺乏必要的注释。Reflection 允许 Agent 充当自己的 Code Reviewer（代码审查员），在提交给用户前自行修复错误并优化代码风格。
- **复杂摘要 (Complex Summarization)**
   在总结密集型文档时，第一遍摘要可能会遗漏细微差别或关键细节。反思步骤有助于进行查漏补缺，确保摘要的全面性和准确性。
- **创意写作与内容创作 (Creative Writing)**
   邮件、博客或故事的初稿往往有提升空间。Reflection 允许 Agent 优化语气、清晰度和感染力，使其更符合预期目标。

---

## 4. 优缺点分析

### 优点
- **质量提升**：通过自我修正，直接解决了初稿中的错误，产出更准确、更稳健且逻辑更严密的回答。
- **实现门槛低 (Low Overhead)**：这是一个概念上非常简单的设计模式，只需要单个 LLM 即可实现，通常不需要引入复杂的外部工具或插件。

### 缺点
- **自我局限 (Self-Bias)**：Agent 仍然受限于自身的知识边界和模型偏见。如果它根本不知道更好的解法，它就无法通过“批评”来得出更优的结果（即：它可以修复它**能识别**的错误，但无法创造它**不知道**的知识）。
- **延迟与成本增加**：该过程至少涉及两次 LLM 调用（生成 + 批评/优化），相比单次生成（One-pass），响应速度更慢，且推理成本更高。

# 0. 环境配置

## 0.1 安装核心依赖
`langchain-openai` 包提供了访问 OpenAI等模型的能力，`langchain` 和 `langgraph` 将提供核心的编排框架，`python-dotenv` 用于管理 API 密钥，而 `rich` 则用于美化终端输出。

**<font color="orange">在Jupyter Notebook里运行pip的时候要添加`%`前缀，一定一定一定不要用`!`前缀，因为`!`会会去找系统的默认 Python。`%`会自动锁定当前 Notebook 正在使用的那个 Python 内核。</font>**

In [ ]:
%pip install langchain-openai langchain langgraph rich python-dotenv pygraphviz

## 0.2 导入依赖并设置密钥
使用 `python-dotenv` 库从本地 `.env` 文件安全地加载API 密钥。同时还将配置 LangSmith 进行追踪（Tracing），这对于非常直观地调试多步骤的 Agent 工作流。

---
这里我使用了阿里百炼大模型平台的模型，因为有免费额度

In [ ]:
import os
import json
from typing import List, TypedDict, Optional
from dotenv import load_dotenv

# OpenAI和LangChain组件
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field # Corrected import for Pydantic v2
from langgraph.graph import StateGraph, END

# 美化输出
from rich.console import Console
from rich.markdown import Markdown
from rich.syntax import Syntax

# --- 密钥加载、追踪配置 ---
load_dotenv()

# 配置LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Reflection (OpenAI)"

# 检查
if not os.environ.get("DASHSCOPE_API_KEY"):
    print("DASHSCOPE_API_KEY not found. Please create a .env file and set it.")
if not os.environ.get("LANGCHAIN_API_KEY"):
    print("LANGCHAIN_API_KEY not found. Please create a .env file and set it for tracing.")

print("Environment variables loaded and tracing is set up.")

# 构建 Reflection

构建健壮的 Reflection 架构的核心思路，重点在于结构化系统的设计与基于 Pydantic 的数据验证。

## 1. 核心架构设计：
一个健壮的 Reflection 架构不仅仅依赖简单的提示词 (Prompt)，而是需要构建一个结构化的三部分系统，各司其职以确保输出质量。

* **Generator**
    * 负责生成初始的回答、代码或解决方案。
    * 作为流程的起点，提供原始素材。
* **Critic**
    * 负责评估 Generator 的输出。
    * 识别潜在的错误、逻辑漏洞或可改进之处。
    * 输出结构化的反馈意见。
* **Refiner**
    * 基于 Critic 的反馈，对初始输出进行修正和优化。
    * 生成最终的高质量结果。

## 2. 数据验证：使用 Pydantic
为了确保多步骤流程的可靠性（即上一步的输出能无缝作为下一步的输入），必须严格定义数据结构。

### 2.1 Pydantic 的作用
* **作为契约 (Contract)**：Pydantic 模型充当了代码与 LLM 之间的协议，明确告知 LLM 必须输出什么样的结构。
* **结构化保障**：这对于多步工作流 (Multi-step process) 至关重要，避免了 LLM 输出自由文本导致的解析错误。

### 2.2 实施策略
* **定义模型**：为 Generator、Critic 和 Refiner 的预期输出分别定义 Pydantic 类。
* **明确字段**：在类中清晰定义字段名称和类型（例如：`draft`, `critique`, `refined`）。
* **自动验证**：利用 Pydantic 自动验证 LLM 的输出是否符合 Schema，确保系统稳定性。

In [ ]:
class DraftCode(BaseModel):
    """代理生成的初始代码草稿的结构."""
    code: str = Field(description="The Python code generated to solve the user's request.")
    explanation: str = Field(description="A brief explanation of how the code works.")

class Critique(BaseModel):
    """对生成的代码进行自我评价的结构。"""
    has_errors: bool = Field(description="Does the code have any potential bugs or logical errors?")
    is_efficient: bool = Field(description="Is the code written in an efficient and optimal way?")
    suggested_improvements: List[str] = Field(description="Specific, actionable suggestions for improving the code.")
    critique_summary: str = Field(description="A summary of the critique.")

class RefinedCode(BaseModel):
    """采纳批评意见后最终精简代码的结构."""
    refined_code: str = Field(description="The final, improved Python code.")
    refinement_summary: str = Field(description="A summary of the changes made based on the critique.")

print("Pydantic models for Draft, Critique, and RefinedCode have been defined.")

Critique 模型（评估模型）的作用至关重要。通过要求其输出特定的字段（如 `has_errors` 和 `is_efficient`），我们能够引导大语言模型（LLM）进行更具结构化且实用性强的评估，这比单纯让它“审查代码”要有效得多。

---

## 1.2 初始化 LLM 与终端环境

初始化负责驱动这三个角色（生成器 Generator、评估器 Critic 和优化器 Refiner）的 QWEN 模型。本示例采用 `qwen-flash` 模型，同时配置了 `rich` 终端控制台，以便输出整洁且格式化的结果。

> [!CAUTION]
> **重要提示：请勿在此处使用推理型模型（Reasoning Models）**
> 推理模型的响应结构通常如下：
> ```python
> {
>     'reasoning_tokens': 172,  # 包含推理思维链过程
>     'parsed': None,
>     'refusal': None
> }
> ```
> 由于推理过程的输出格式往往会与预设的结构化格式产生冲突，因此不支持直接输出 Structured Outputs。

In [ ]:
llm = ChatOpenAI(
    model="qwen-flash",
    openai_api_key=os.environ["DASHSCOPE_API_KEY"],
    openai_api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
    temperature=0.1
)

console = Console()

print("LLM and console initialized.")

## 1.3 创建代码生成节点

该节点的核心职责是接收用户请求并生成代码初稿。为了确保输出结果的结构化与准确性，将 `DraftCode` Pydantic 模型与 QWEN 大模型进行绑定，从而强制规范其生成的响应格式。

In [ ]:
def generate_node(state):
    """生成代码的初版"""
    console.print("--- 1. 生成初始代码 ---")
    generator_llm = llm.with_structured_output(DraftCode)

    prompt = f"""You are an expert Python programmer. Write a Python function to solve the following request.
    Provide a simple, clear implementation and an explanation.

    Request: {state['user_request']}
    """
    draft = generator_llm.invoke(prompt)
    return {"draft": draft.model_dump()}

## 1.4 创建代码评价节点

这是反思过程（Reflection Process）的核心环节。评价节点（Critic node）会接收生成的代码初稿，分析其中的缺陷，并基于预定义的 `Critique` Pydantic 模型生成结构化的评价报告。

In [ ]:
def critic_node(state):
    """对生成的代码进行自我评价"""
    console.print("--- 2. 评价初始代码 ---")
    critic_llm = llm.with_structured_output(Critique)

    code_to_critique = state["draft"]["code"]

    prompt = f"""You are an expert code reviewer and senior Python developer. Your task is to perform a thorough critique of the following code.

    Analyze the code for:
    1.  **Bugs and Errors:** Are there any potential runtime errors, logical flaws, or edge cases that are not handled?
    2.  **Efficiency and Best Practices:** Is this the most efficient way to solve the problem? Does it follow standard Python conventions (PEP 8)?

    Provide a structured critique with specific, actionable suggestions.

    Code to Review:
    ```python
    {code_to_critique}
    ```
    """

    critique = critic_llm.invoke(prompt)
    return {"critique": critique.model_dump()}

## 1.5 创建代码改进节点

该节点同时接收原始代码初稿与结构化的评价反馈，其任务是根据这些信息编写最终的、经过优化的改进版代码。

In [ ]:
def refine_node(state):
    """对代码进行改进"""
    console.print("--- 3. 完善代码 ---")
    refiner_llm = llm.with_structured_output(RefinedCode)

    draft_code = state["draft"]["code"]
    critique = json.dumps(state["critique"], indent=2)

    prompt = f"""You are an expert Python programmer tasked with refining a piece of code based on a critique.

    Your goal is to rewrite the original code, implementing all the suggested improvements from the critique.

    **Original Code:**
    ```python
    {draft_code}
    ```

    **Critique and Suggestions:**
    {critique}

    Please provide the final, refined code and a summary of the changes you made.
    """

    refined_code = refiner_llm.invoke(prompt)
    return {"refined_code": refined_code.model_dump()}

现已完成了Reflective Agent的三个核心逻辑组件。每个组件都是一个功能独立的函数（或称为“节点”），负责执行单一且明确的任务。在每个阶段使用结构化输出，确保了数据在节点间能够可靠地传递。接下来使用 LangGraph 来编排这一工作流。

# 2. 用 LangGraph 编排反射工作流

## 2.1 定义 LangGraph 状态（State）

“状态（State）”是图结构（Graph）的记忆中枢。它是一个在节点之间传递的核心对象，每个节点都可以对其进行读取或写入。我们将使用 Python 的 `TypedDict` 来定义 `ReflectionState`，用以承载工作流中的所有数据片段。

In [ ]:
class ReflectionState(TypedDict):
    """Reflection图的状态"""
    user_request: str
    draft: Optional[dict]
    critique: Optional[dict]
    refined_code: Optional[dict]

print("ReflectionState defined.")

## 2.2 构建并可视化图（Graph）

使用 `StateGraph` 将之前定义的各个节点组装成一个连贯的工作流。在这一Reflection Pattern中，工作流遵循一个简单的线性序列：**生成（Generate） → 评价（Critique） → 改进（Refine）**。接下来将定义这一流转逻辑，随后进行编译并对图结构进行可视化，以确认其架构是否符合预期。

In [ ]:
graph_builder = StateGraph(ReflectionState)

# 在图里加上节点
graph_builder.add_node("generator", generate_node)
graph_builder.add_node("critic", critic_node)
graph_builder.add_node("refiner", refine_node)

# 定义工作流
graph_builder.set_entry_point("generator")
graph_builder.add_edge("generator", "critic")
graph_builder.add_edge("critic", "refiner")
graph_builder.add_edge("refiner", END)

# 编译图结构
reflection_graph = graph_builder.compile()

print("Graph build successfully")

# 可视化图结构
try:
    from IPython.display import display, Image
    png_image = reflection_graph.get_graph().draw_png()
    display(Image(png_image))
except Exception as e:
    print(f"Graph visualization failed: {e}. Please ensure pygraphviz is installed.")

可视化结果证实了预期的线性工作流。可以清晰地看到状态（State）是如何从入口点（生成器 generator）开始，依次流经评价节点（critic）和改进节点（refiner），最终到达结束状态（__end__）。这一简单而强大的结构现已准备就绪，可以正式执行。

# 3. 端到端的执行与Eval

将交给智能体一个编程任务。对于该任务，初次尝试往往会得到一个次优方案，这使其成为测试“自我评价与改进”能力的绝佳案例。

## 3.1 运行完整的反射工作流

将调用编译好的 LangGraph 应用，请求其编写一个高性能的 Python 排序函数并包含相应的测试用例。采用流式输出（Stream）的方式获取结果，并完整地累积最终状态，以便在流程结束时审查所有的中间步骤。

In [ ]:
user_request = "Write a fast sorting and a test case for it."
initial_input = {"user_request": user_request}

console.print(f"[bold cyan]🚀 开始针对请求进行Reflection工作流编排:[/bold cyan] '{user_request}'\n")

final_state = None
# 流式输出
for state in reflection_graph.stream(initial_input, stream_mode="values"):
    final_state = state

console.print("\n[bold green]✅ Reflection工作流完成![/bold green]")

## 3.2 分析Reflection前后代码对比

这一步详细检查存储在 `final_state` 中的工作流各阶段输出结果。通过对比打印出的代码初稿、收到的评价反馈以及最终的改进版代码，可以清晰地观察到“反射过程”为代码质量带来的显著价值提升。

In [ ]:
# 检查final_state并查看是否有期望的键值
if final_state and 'draft' in final_state and 'critique' in final_state and 'refined_code' in final_state:
    console.print(Markdown("--- ### 初版代码 ---"))
    console.print(Markdown(f"**注释:** {final_state['draft']['explanation']}"))
    # 使用rich库里的Syntax来高亮显示代码
    console.print(Syntax(final_state['draft']['code'], "python", theme="monokai", line_numbers=True))
    console.print(Markdown("\n--- ### 代码修改意见 ---"))
    console.print(Markdown(f"**概要:** {final_state['critique']['critique_summary']}"))
    console.print(Markdown(f"**修改意见:**"))
    for improvement in final_state['critique']['suggested_improvements']:
        console.print(Markdown(f"- {improvement}"))

    console.print(Markdown("\n--- ### 完善后的代码 ---"))
    console.print(Markdown(f"**完善概要:** {final_state['refined_code']['refinement_summary']}"))
    console.print(Syntax(final_state['refined_code']['refined_code'], "python", theme="monokai", line_numbers=True))
else:
    console.print("[bold red]Error: `final_state`不可用或不完整. 检查之前的执行[/bold red]")

## 3.3 量化Eval

为了使分析更加正式化，将引入另一个大语言模型（LLM）作为公正的“评判者（Judge）”，对代码初稿与最终版本的质量分别进行打分。这为通过反射模式获得的性能提升提供了一个更加客观的衡量标准。

In [ ]:
class CodeEvaluation(BaseModel):
    """Schema for evaluating a piece of code."""
    correctness_score: int = Field(description="判断代码逻辑是否正确，评分范围为 1-10 分。")
    efficiency_score: int = Field(description="Score from 1-10 on the code's algorithmic efficiency.")
    style_score: int = Field(description="对代码的算法效率进行评分，范围为 1-10 分。 ")
    justification: str = Field(description="对评分的简要说明。")

judge_llm = llm.with_structured_output(CodeEvaluation)

def evaluate_code(code_to_evaluate: str):
    prompt = f"""您是一位Python代码专家，请根据正确性、效率和风格，以1-10分评价以下函数，并简要说明理由。

    Code:
    ```python
    {code_to_evaluate}
    ```
    """
    return judge_llm.invoke(prompt)

if final_state and 'draft' in final_state and 'refined_code' in final_state:
    console.print("--- 评估初稿 ---")
    initial_draft_evaluation = evaluate_code(final_state['draft']['code'])
    console.print(initial_draft_evaluation.model_dump()) # Corrected: use .model_dump()

    console.print("\n--- 评估终稿 ---")
    refined_code_evaluation = evaluate_code(final_state['refined_code']['refined_code'])
    console.print(refined_code_evaluation.model_dump()) # Corrected: use .model_dump()
else:
    console.print("[bold red]Error: Cannot perform evaluation because the `final_state` is incomplete.[/bold red]")

“LLM-as-a-Judge”（以大模型作为评判者）的评估方法，为反射模式的成功提供了量化的证据。

在评估中，初稿代码在正确性（Correctness）上可能获得了高分，但在执行效率（Efficiency）上的得分通常很低。相比之下，经过优化后的代码在正确性和效率上都能获得高分。这种自动化的评分评估证实了反射过程不仅仅是“修改”了代码，而是以一种可衡量的方式显著“提升”了代码质量。

# 结论

在本指南中，用了阿里的百炼大模型模型，基于Reflection Architecture成功构建、执行并评估了一个完整的端到端智能体。这种简单而强大的模式可以将一个基础的 LLM 生成器转变为一个更成熟、更可靠的问题解决者。

通过将开发流程划分为明确的“生成（Generate）”、“评价（Critique）”和“改进（Refine）”步骤，并使用 LangGraph 进行编排，我们构建了一个能够识别并纠正自身重大缺陷的健壮系统。